#**컨텐츠 기반 추천 시스템**
####컨텐츠 기반 추천시스템은 사용자가 이전에 구매한 상품중에서 좋아하는 상품들과 유사한 상품들을 추천하는 방법. 이때의 컨텐츠(Items)들을 벡터 형태로 표현해서 유사도를 확인함

#####추천시스템의 장점
1.   다른 유저의 데이터가 필요하지 않다.
2.   추천할 수 있는 아이템의 범위가 넓다. 즉 새로운 아이템이나 인기 없는 아이템도 추천이 가능한다.
1.   추천하는 이유를 제시할 수 있다. 아이템의 features로 컨텐츠를 분석하는 것이 가능하기 때문에, 특정 feature가 추천의 이유가 됐다고 설명이 가능하다.

#####컨텐츠 기반 추천시스템의 단점
1.   적절한 features를 찾기가 어렵다.
2.   새로운 유저를 위한 추천이 어렵다. 특, 유저 프로필이 존재하지 않거나 데이터가 매우 부족한 경우에 해당 유저에게 추천시스템을 적용하기 어렵다.
1.   선호하는 특성을 가진 항목을 반복 추천한다. 즉 추천시스템이 overspecialization되는 경향이 있으며, 유저의 다양한 취향을 반영하기 어렵다. 




#텐서플로우 레코멘더 설치 
텐서플로우 추천시스템 생성을 위한 라이브러리 설치

In [ ]:
pip install tensorflow-recommenders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings

from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import jaccard_similarity_score
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [ ]:
data_19=pd.read_csv('/content/drive/MyDrive/project/MIB/df2019_preprocess_0820_re.csv')
data_19

In [ ]:
data_19.drop('Unnamed: 0', axis=1, inplace=True)
data_19=data_19[['name','f_name','track_length','top_speed','lowest_ele','top_ele','moving_time','elevation_gain','Kcal','sports_type','sex','age','season','time_div']]
data_19.head()

#**코사인 유사도**
####**무게나 키 라는 x, y축 데이터를 고려하지 않고 두 벡터간의 각도로만 유사도를 측정하는 것. 일반적으로 벡터의 크기가 중요하지 않을 때 거리를 측정하기 위한 방법으로 사용**
https://docs.likejazz.com/cosine-sim/ 참고! <br><br>

+2차원 배열로 변경해준 이유<br>
-Expected 2D array, got 1D array instead 에러 때문에. sklearn은 거의 모든 경우에서 자신의 인풋 X가 2d로 들어 올 것으로 상정하고 있음.<br>
근데 여기다가 1d array 또는 list 를 주면? 이게 하나의 feature에 여러개의 samples이 있는건지, 여러개의 features에 하나씩의 sample이 있는건지 알 수가 없다. 그래서 오류가 발생하게됨<br>
출처: https://datamasters.co.kr/55 [데이터마스터]

+왜 우리는 벡터화 안함?<br>
-사실 응용한 코드에서 사용하지 않아서 안쓴 거긴 합니다..... 근데 내 생각에는 이미 수치화된 데이터라 벡터화할 필요성이 없어서 안한 것 같다


In [ ]:
def cos_sim_recommendations(hike_idx, df, index_name, n=5):
    hike = df.iloc[hike_idx].to_numpy().reshape(1, -1)# 테스트 예시를 2차원 배열로 변경
    cs = cosine_similarity(hike, df)# 코사인 유사도 체크
    rec_index = np.argsort(cs)[0][-6:][::-1][1:]# 정렬, 근데 왜 이렇게 짰는지는 모르겠음
    recommendations = []
    for rec in rec_index:#인덱스에 따라 파일명 추출
        recommendations.append(index_name[rec])
    return recommendations

if __name__ == '__main__':
    d1=data_19[['f_name','track_length','top_speed','lowest_ele','top_ele','moving_time','elevation_gain','Kcal','sports_type','sex','age','season','time_div']]
    index_name = d1['f_name'].values  #gpx이름 컬럼을 리스트로 추출하여 저장
    d1.drop('f_name', axis=1, inplace=True)# 'gpx이름 컬럼 제거(숫자로 변경되지 않은 항목 제거)

    hike_name = 'done_997163.gpx'#테스트 예시
    for idx, name in enumerate(index_name):#테스트 예시의 인덱스 추출
        if name == hike_name:
            hike_idx = idx
    print()
    recommendations = cos_sim_recommendations(hike_idx, d1, index_name, n=5)# 코사인 유사도 비교 함수

#테스트 예시 입력, 결과물 출력
print('선택예시:',hike_name)
print(data_19.iloc[hike_idx])
print('================================')
for n in recommendations:
      for idx, name in enumerate(index_name):
        if name == n:
          print(n)
          print(data_19.iloc[idx])
          print()
          break

# **같은 스타팅 포인트가 아닌 경우**

In [ ]:
# 조건를 충족하는 데이터를 필터링
hike_name2='done_1000362.gpx'
sele = data_19[data_19['f_name']==hike_name2]
sele_point=sele[['track_length','top_speed','lowest_ele','top_ele','moving_time','elevation_gain','Kcal','sports_type','sex'	,'age',	'season',	'time_div']]


#다른 스타팅 포인트를 지닌 데이터프레임 생성
start_p=sele['name'].values[0]
target=[start_p]
d2= data_19.loc[~data_19['name'].isin(target)]
d2=d2[['f_name','track_length','top_speed','lowest_ele','top_ele','moving_time','elevation_gain','Kcal','sports_type','sex'	,'age',	'season',	'time_div']]
d2

In [ ]:
def cos_sim_recommendations2(sele_point, df_sle,index_name, n=5):
    hike = sele_point.to_numpy().reshape(1, -1)# 테스트 예시를 2차원 배열로 변경
    cs = cosine_similarity(hike, df_sle)# 코사인 유사도 체크
    rec_index = np.argsort(cs)[0][-6:][::-1][1:]# 정렬, 근데 왜 이렇게 짰는지는 모르겠음
    recommendations = []
    for rec in rec_index:#인덱스에 따라 파일명 추출
        recommendations.append(index_name[rec])
    return recommendations

if __name__ == '__main__':
    index_name = data_19['f_name'].values  #gpx이름 컬럼을 리스트로 추출하여 저장
    d2.drop('f_name', axis=1, inplace=True)# 'gpx이름 컬럼 제거(숫자로 변경되지 않은 항목 제거)

    recommendations = cos_sim_recommendations2(sele_point, d2,index_name, n=5)# 코사인 유사도 비교 함수

#테스트 예시 입력, 결과물 출력
print('선택예시:',hike_name2)
print(data_19.iloc[hike_idx])
print('================================')
for n in recommendations:
      for idx, name in enumerate(index_name):
        if name == n:
          print(n)
          print(data_19.iloc[idx])
          print()
          break

#**같은 스타팅 포인트 내부에서 추천할 경우**

In [ ]:
# 조건를 충족하는 데이터를 필터링
hike_name3='done_1000237.gpx'
sele = data_19[data_19['f_name']==hike_name3]
sele_point=sele[['track_length','top_speed','lowest_ele','top_ele','moving_time','elevation_gain','Kcal','sports_type','sex'	,'age',	'season',	'time_div']]


#다른 스타팅 포인트를 지닌 데이터프레임 생성
start_p=sele['name'].values[0]
target=[start_p]
d3= data_19.loc[data_19['name'].isin(target)]
d3=d3[['f_name','track_length','top_speed','lowest_ele','top_ele','moving_time','elevation_gain','Kcal','sports_type','sex'	,'age',	'season',	'time_div']]
d3

In [ ]:
def cos_sim_recommendations3(sele_point, df_sle,index_name, n=5):
    hike = sele_point.to_numpy().reshape(1, -1)# 테스트 예시를 2차원 배열로 변경
    cs = cosine_similarity(hike, df_sle)# 코사인 유사도 체크
    rec_index = np.argsort(cs)[0][-6:][::-1][1:]# 정렬, 근데 왜 이렇게 짰는지는 모르겠음
    recommendations = []
    for rec in rec_index:#인덱스에 따라 파일명 추출
        recommendations.append(index_name[rec])
    return recommendations

if __name__ == '__main__':
    index_name = data_19['f_name'].values  #gpx이름 컬럼을 리스트로 추출하여 저장
    d3.drop('f_name', axis=1, inplace=True)# 'gpx이름 컬럼 제거(숫자로 변경되지 않은 항목 제거)

    recommendations = cos_sim_recommendations3(sele_point, d3,index_name, n=5)# 코사인 유사도 비교 함수

#테스트 예시 입력, 결과물 출력
print('선택예시:',hike_name2)
print(data_19.iloc[hike_idx])
print('================================')
for n in recommendations:
      for idx, name in enumerate(index_name):
        if name == n:
          print(n)
          print(data_19.iloc[idx])
          print()
          break